# Validación de Ortogonalidad

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

## Sección 1: Simular variables (para un ejemplo ejecutable)

A diferencia del Notebook anterior, ya tienes cov_matrix de tus datos (X_std). Aquí creamos una matriz de covarianza simulada 30x30

In [ ]:
np.random.seed(42)
simulated_data = np.random.rand(100, 30)

También necesitamos los datos estandarizados para la parte 2

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(simulated_data)

cov_matrix = np.cov(X_std, rowvar=False)
eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)
print("Variables 'eigen_vectors' y 'eigen_values' listas.\n")

Variables 'eigen_vectors' y 'eigen_values' listas.



### 1. Validación de Ortogonalidad (Producto Punto de Vectores Propios)

Extraemos los dos primeros vectores propios (Componentes Principales).

**Nota:** `np.linalg.eig` devuelve los vectores como COLUMNAS


In [ ]:
pc1_vector = eigen_vectors[:, 0]
pc2_vector = eigen_vectors[:, 1]

#### 1.1. Validar Ortogonalidad:

El producto punto entre dos vectores propios DIFERENTES debe ser 0

In [ ]:
dot_product_1_2 = np.dot(pc1_vector, pc2_vector)
print(f"Producto punto entre PC1 y PC2: {dot_product_1_2}")

Producto punto entre PC1 y PC2: 1.3877787807814457e-17


#### 1.2. Validar Normalidad (Ortonormalidad):

El producto punto de un vector consigo mismo debe ser 1 (porque son vectores unitarios)

In [ ]:
dot_product_1_1 = np.dot(pc1_vector, pc1_vector)
print(f"Producto punto entre PC1 y PC1: {dot_product_1_1}")

print("\nExplicación (Producto Punto):")
print(f"El producto punto entre PC1 y PC2 es un número muy cercano a 0 (ej: {dot_product_1_2:.2e}).")
print("No es exactamente 0 debido a la precisión de punto flotante de la computadora.")
print(f"El producto punto de PC1 consigo mismo es {dot_product_1_1:.1f}, lo que confirma que es un vector unitario.")


Producto punto entre PC1 y PC1: 1.0

Explicación (Producto Punto):
El producto punto entre PC1 y PC2 es un número muy cercano a 0 (ej: 1.39e-17).
No es exactamente 0 debido a la precisión de punto flotante de la computadora.
El producto punto de PC1 consigo mismo es 1.0, lo que confirma que es un vector unitario.


Prueba de cercanía a cero

In [ ]:
is_orthogonal = np.isclose(dot_product_1_2, 0)
print(f"¿Son PC1 y PC2 ortogonales (matemáticamente)? {is_orthogonal}")

¿Son PC1 y PC2 ortogonales (matemáticamente)? True


### 2. Validación de Correlación (Datos Transformados)

Transformamos los datos originales (`X_std`) usando TODOS los eigenvectors
$Z = X * W$ (donde $W$ es la matriz de eigenvectors)

In [ ]:
Z_pca = X_std.dot(eigen_vectors)

Creamos un DataFrame de pandas para calcular la correlación fácilmente y nombramos las columnas PC_1, PC_2, etc.

In [ ]:
pc_columns = [f'PC_{i+1}' for i in range(eigen_vectors.shape[1])]
pca_df = pd.DataFrame(Z_pca, columns=pc_columns)

Calculamos la matriz de correlación

In [ ]:
corr_matrix = pca_df.corr()

Mostramos solo los primeros 5x5 para que sea legible

In [ ]:
print("Matriz de Correlación de los Componentes Principales (primeros 5):")
print(corr_matrix.iloc[:5, :5].to_string(float_format="%.2e"))

Matriz de Correlación de los Componentes Principales (primeros 5):
          PC_1      PC_2      PC_3      PC_4      PC_5
PC_1  1.00e+00 -2.48e-17  6.93e-16  6.74e-16 -9.37e-16
PC_2 -2.48e-17  1.00e+00 -2.29e-15 -7.91e-16  1.50e-16
PC_3  6.93e-16 -2.29e-15  1.00e+00 -4.24e-15 -2.50e-15
PC_4  6.74e-16 -7.91e-16 -4.24e-15  1.00e+00 -3.55e-16
PC_5 -9.37e-16  1.50e-16 -2.50e-15 -3.55e-16  1.00e+00


**Explicación (Correlación):**

Como puedes ver, la diagonal es 1 (cada componente correlaciona 100% consigo mismo). Así mismo, todos los demás valores (fuera de la diagonal) son 0 o muy cercanos a 0 (ej: -3.85e-17). Esto confirma que los nuevos componentes están DESCORRELACIONADOS (que es el objetivo del PCA).


## Sección 2: Validación con el dataset Breast Cancer

Cargamos y preparamos los datos reales

In [26]:
cancer = load_breast_cancer()
data_cancer = cancer.data

Estandarizamos los datos

In [ ]:
scaler_cancer = StandardScaler()
X_std_cancer = scaler_cancer.fit_transform(data_cancer)

Calculamos la matriz de covarianza y los eigen-vectores/valores

In [28]:
cov_matrix_cancer = np.cov(X_std_cancer, rowvar=False)
eigen_values_cancer, eigen_vectors_cancer = np.linalg.eig(cov_matrix_cancer)

### 1. Validación de Ortogonalidad (Producto Punto) - CÁNCER

Extraemos los dos primeros vectores propios

In [29]:
pc1_vector_cancer = eigen_vectors_cancer[:, 0]
pc2_vector_cancer = eigen_vectors_cancer[:, 1]

#### 1.1. Validar Ortogonalidad:

In [30]:
dot_product_1_2_cancer = np.dot(pc1_vector_cancer, pc2_vector_cancer)
print(f"Producto punto entre PC1 y PC2: {dot_product_1_2_cancer}")

Producto punto entre PC1 y PC2: 2.949029909160572e-16


#### 1.2. Validar Normalidad (Ortonormalidad):

In [31]:
dot_product_1_1_cancer = np.dot(pc1_vector_cancer, pc1_vector_cancer)
print(f"Producto punto entre PC1 y PC1: {dot_product_1_1_cancer}")

Producto punto entre PC1 y PC1: 1.0


In [32]:
print("\nExplicación (Producto Punto):")
print(f"El producto punto entre PC1 y PC2 es {dot_product_1_2_cancer:.2e} (cercano a 0).")
print(f"El producto punto de PC1 consigo mismo es {dot_product_1_1_cancer:.1f} (cercano a 1).")


Explicación (Producto Punto):
El producto punto entre PC1 y PC2 es 2.95e-16 (cercano a 0).
El producto punto de PC1 consigo mismo es 1.0 (cercano a 1).


Prueba de cercanía a cero

In [33]:
is_orthogonal_cancer = np.isclose(dot_product_1_2_cancer, 0)
print(f"¿Son PC1 y PC2 ortogonales (matemáticamente)? {is_orthogonal_cancer}")

¿Son PC1 y PC2 ortogonales (matemáticamente)? True


### 2. Validación de Correlación (Datos Transformados) - CÁNCER

Transformamos los datos de cáncer (X_std_cancer)

In [34]:
Z_pca_cancer = X_std_cancer.dot(eigen_vectors_cancer)

In [ ]:
# Creamos un DataFrame de pandas
pc_columns_cancer = [f'PC_{i+1}' for i in range(eigen_vectors_cancer.shape[1])]
pca_df_cancer = pd.DataFrame(Z_pca_cancer, columns=pc_columns_cancer)


In [ ]:

# Calculamos la matriz de correlación
corr_matrix_cancer = pca_df_cancer.corr()

Matriz de Correlación de los Componentes (primeros 5). Mostramos solo los primeros 5x5


In [35]:
print(corr_matrix_cancer.iloc[:5, :5].to_string(float_format="%.2e"))

          PC_1      PC_2      PC_3      PC_4     PC_5
PC_1  1.00e+00  4.25e-16 -3.69e-16 -2.64e-16 2.23e-16
PC_2  4.25e-16  1.00e+00  4.00e-17 -3.81e-17 6.29e-17
PC_3 -3.69e-16  4.00e-17  1.00e+00  4.07e-16 1.84e-16
PC_4 -2.64e-16 -3.81e-17  4.07e-16  1.00e+00 9.78e-16
PC_5  2.23e-16  6.29e-17  1.84e-16  9.78e-16 1.00e+00


**Explicación (Correlación):**

Nuevamente, la diagonal es 1 y los demás valores son prácticamente 0. Esto confirma que la descorrelación funciona igual de bien en un dataset real.